In [1]:
import sys
sys.path.append("..")
sys.path.append("../env")

from env.Cylinder_Rotation_Env import Cylinder_Rotation_Env
import numpy as np
import torch
from timeit import default_timer

import matplotlib.pyplot as plt
from fenics import plot

In [3]:
# env init
env = Cylinder_Rotation_Env(params={'dtr': 0.1, 'T': 1, 'rho_0': 1, 'mu' : 1/1000,
                                    'traj_max_T': 20, 'dimx': 128, 'dimy': 64,
                                    'min_x' : 0,  'max_x' : 2.2, 
                                    'min_y' : 0,  'max_y' : 0.41, 
                                    'r' : 0.05,  'center':(0.2, 0.2),
                                    'min_w': -1, 'max_w': 1,
                                    'min_velocity': -1, 'max_velocity': 1,
                                    'U_max': 1.5, })

print(env.params)

# one step: 10 timestamps for dT = 1

# env params
dT = env.params['T']
nx = env.params['dimx']
ny = env.params['dimy']
print('dT: {}, ny: {}, nx: {}'.format(dT, ny, nx))

/opt/conda/envs/pytorch-py3.8/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


start init_solve
end init_solve
{'dtr': 0.1, 'T': 1, 'rho_0': 1, 'mu': 0.001, 'traj_max_T': 20, 'dimx': 128, 'dimy': 64, 'min_x': 0, 'max_x': 2.2, 'min_y': 0, 'max_y': 0.41, 'r': 0.05, 'center': (0.2, 0.2), 'min_w': -1, 'max_w': 1, 'min_velocity': -1, 'max_velocity': 1, 'U_max': 1.5}
dT: 1, ny: 64, nx: 128


# Phase 1

In [4]:
# data generation

N0 = 25     # N0 set of data
nT = 10
print(f'dT: {dT} | nT: {nT}')

# data generate
obs = np.zeros((N0, nT+1, nx, ny, 3))
print(f'state_data_size :{obs.shape}')
C_D, C_L, reward = np.zeros((N0, nT)), np.zeros((N0, nT)), np.zeros((N0, nT))
ang_vel = 3 * (2 * np.random.rand(N0, nT) - 1)
for k in range(N0):
    start = default_timer()
    obs[k, 0] = env.reset()
    
    ang_v = ang_vel[k]
    
    for i in range(nT):
        obs[k, i+1], reward[k, i], C_D[k, i], C_L[k, i] = env.step(ang_v[i])   
    
    end = default_timer()

    print(f'# {k} | time: {end-start}')

# np to tensor
obs_tensor = torch.Tensor(obs)
reward_tensor = torch.Tensor(reward)
C_D_tensor = torch.Tensor(C_D)
C_L_tensor = torch.Tensor(C_L)
ang_vel_tensor = torch.Tensor(ang_vel)

data = [obs_tensor, reward_tensor, C_D_tensor, C_L_tensor, ang_vel_tensor]

# save data
torch.save(data, './data/nse_data_N0_{}_nT_{}'.format(N0, nT))

dT: 1 | nt: 10
state_data_size :(25, 11, 128, 64, 3)
# 0 | time: 109.7380578420125
# 1 | time: 109.7899524946697
# 2 | time: 109.9847599635832
# 3 | time: 111.28753285389394
# 4 | time: 110.35802854411304


# Phase 2

In [ ]:
# data generation

N0 = 1     # N0 set of data
nT = 10

# data generate
obs = np.zeros((N0, nT+1, nx, ny, 3))
print(f'state_data_size :{obs.shape}')
C_D, C_L, reward = np.zeros((N0, nT)), np.zeros((N0, nT)), np.zeros((N0, nT))
ang_vel = 3 * (2 * np.random.rand(N0, nT) - 1)
for k in range(N0):
    start = default_timer()
    obs[k, 0] = env.reset()
    
    ang_v = ang_vel[k]
    
    for i in range(nT):
        # obs, reward, C_D, C_L, episode_over, _ = env.step(ang_vel[i])
        obs[k, i+1], reward[k, i], C_D[k, i], C_L[k, i] = env.step(ang_v[i])
        # obs[k, i], reward[k, i], C_D[k, i], C_L[k, i] = env.step(0.00)     
    
    end = default_timer()

    print(f'# {k} | time: {end-start}')
    # print(f'ang_vel: {ang_v}')
    # print(f'reward :{reward[k]}')

# np to tensor
obs_tensor = torch.Tensor(obs)
reward_tensor = torch.Tensor(reward)
C_D_tensor = torch.Tensor(C_D)
C_L_tensor = torch.Tensor(C_L)
ang_vel_tensor = torch.Tensor(ang_vel)

data = [obs_tensor, reward_tensor, C_D_tensor, C_L_tensor, ang_vel_tensor]

# save data
torch.save(data, './data/nse_control_samples')

[[0.7489629  0.8306361  0.81679702 0.64059824 0.14468882 0.6300126
  0.54851793 0.36947325 0.94468961 0.94393485]]


# Plot show

In [ ]:
# load data
data = data, _, Cd, Cl, ang_vel = torch.load('data/nse_data_N0_100_nT_100')

# data param
ny = data.shape[2] 
nx = data.shape[3]
s = data.shape[2] * data.shape[3]     # ny * nx
N0 = data.shape[0]                    # num of data sets
nt = data.shape[1] - 1             # nt

env.reset()

for i in range(nt):
    f = ang_vel[i]
    obs, reward, Cd, Cl = env.step(f)
    env.draw()
